second draft, gonna cut out the mass estimate and:

ignore everything without a mass
if no uncertainty on mass, ignore, 
maybe with radius - ignore too
can calc orbsmax with stellar mass and period
add in transit duration

In [2]:
#initial filtering code
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('Data/composite_exo_data-unfiltered.csv', comment='#')


# Functions to calculate equilibrium temperature and TSM
def calc_Teq(T_star, R_star, a_AU, A=0):
    """Calculates eq temp (assuming full heat redistribution)
    T_star : stellar temp in Kelvin
    R_star : stellar R in R_sun
    a_AU   : orbital semi-major axis [AU]
    A      : bond albedo (default 0)
    """
    a_Rsun = a_AU * 215.032  # need matching units
    return T_star * (R_star / (2*a_Rsun))**0.5 * (1 - A)**0.25

def calc_TSM(R_p, M_p, T_eq, R_star, mJ):
    """Calculates Transmission Spectroscopy Metric (TSM) from Kempton+2018
    R_p : planet radius (R_earth)
    M_p : planet mass (M_earth)
    T_eq: planet eq temp in K
    R_star: units of R_sun
    mJ  : J-band magnitude of host star 
    scale_factor: depends on planet size regime (See Kempton+2018 for some values)
    """

    SF = np.where(R_p < 1.5, 0.190,
        np.where(R_p < 2.75, 1.26, 1.28))
    
    TSM = (SF * (R_p**3 * T_eq) / (M_p * R_star**2) * 10**(-mJ/5))
    return TSM


In [3]:
### GENERAL FILTERING CODE 
# choose columns we want to keep 
useful_columns_only = df[['pl_name', 'pl_orbper', 'pl_orbsmax', 
                'pl_bmasse', 'pl_rade', 'pl_radelim', 'pl_radeerr1', 'pl_radeerr2',
                'pl_trandep', 'pl_trandur', 'pl_bmasselim', 'pl_bmasseerr2', 
                'pl_bmasseerr1', 'pl_orbeccen', 'pl_orbincl', 'st_rad', 
                'st_mass', 'st_teff', 'pl_ratror', 'st_met', 'st_metlim',
                'sy_jmag', 'discoverymethod','pl_controv_flag','st_logg',
                #adding columns tamzin used in her calcs
                'st_tefferr1', 'st_tefferr2',
                'st_raderr1',  'st_raderr2',
                'pl_orbsmaxerr1', 'pl_orbsmaxerr2',
                'sy_jmagerr1', 'sy_jmagerr2']]

default_planets = useful_columns_only.copy()

#remove planets with no mass or 0 uncertainty on mass or lim flag on mass
default_planets_m = default_planets[
    (default_planets['pl_bmasse'].notna()) &
    (default_planets['pl_bmasse'] < 10) &
    (default_planets['pl_orbsmax'].notna()) &
    (default_planets['pl_bmasseerr1'].notna()) &
    (default_planets['pl_bmasseerr2'].notna()) &
    (default_planets['pl_bmasseerr1'] != 0) &
    (default_planets['pl_bmasseerr2'] != 0) &
    (default_planets['pl_bmasselim'] == 0)
]

#remove planets with no radius or 0 uncertainty on radius or lim flag on radius
default_planets_mr = default_planets_m[
    (default_planets_m['pl_rade'].notna()) &
    (default_planets_m['pl_radeerr1'].notna()) &
    (default_planets_m['pl_radeerr2'].notna()) &
    (default_planets_m['pl_radeerr1'] != 0) &
    (default_planets_m['pl_radeerr2'] != 0) &
    (default_planets_m['pl_radelim'] == 0)
]

# remove planets that don't have the necessary parameters for team 1
filtered_planets = default_planets_mr[
    (default_planets_mr['st_rad'].notna()) &
    (default_planets_mr['st_mass'].notna()) &
    (default_planets_mr['st_teff'].notna()) &
    (default_planets_mr['sy_jmag'].notna()) &
    (default_planets_mr['st_met'].notna()) &
    (default_planets_mr['pl_controv_flag'].notna())
]




# filter for the data we need for uncertainty calculations
filtered_planets = filtered_planets[
    (filtered_planets['st_tefferr1'].notna()) & (filtered_planets['st_tefferr2'].notna()) &
    (filtered_planets['st_raderr1'].notna())  & (filtered_planets['st_raderr2'].notna()) &
    (filtered_planets['pl_orbsmaxerr1'].notna()) & (filtered_planets['pl_orbsmaxerr2'].notna()) &
    (filtered_planets['sy_jmagerr1'].notna()) & (filtered_planets['sy_jmagerr2'].notna()) &
    (filtered_planets['st_tefferr1'] != 0) & (filtered_planets['st_tefferr2'] != 0) &
    (filtered_planets['st_raderr1'] != 0)  & (filtered_planets['st_raderr2'] != 0) &
    (filtered_planets['pl_orbsmaxerr1'] != 0) & (filtered_planets['pl_orbsmaxerr2'] != 0) &
    (filtered_planets['sy_jmagerr1'] != 0) & (filtered_planets['sy_jmagerr2'] != 0)
]


# adding columns
# add mean errors
filtered_planets = filtered_planets.copy()

filtered_planets['pl_bmasseerr_avg'] = (
    filtered_planets['pl_bmasseerr1'].abs() + filtered_planets['pl_bmasseerr2'].abs()
) / 2
filtered_planets['pl_radeerr_avg'] = (
    filtered_planets['pl_radeerr1'].abs() + filtered_planets['pl_radeerr2'].abs()
) / 2
filtered_planets['st_tefferr_avg'] = (
    filtered_planets['st_tefferr1'].abs() + filtered_planets['st_tefferr2'].abs()
) / 2
filtered_planets['st_raderr_avg'] = (
    filtered_planets['st_raderr1'].abs() + filtered_planets['st_raderr2'].abs()
) / 2
filtered_planets['pl_orbsmaxerr_avg'] = (
    filtered_planets['pl_orbsmaxerr1'].abs() + filtered_planets['pl_orbsmaxerr2'].abs()
) / 2
filtered_planets['sy_jmagerr_avg'] = (
    filtered_planets['sy_jmagerr1'].abs() + filtered_planets['sy_jmagerr2'].abs()
) / 2


# Constants in cgs
G = 6.67430e-8   # cm^3 g^-1 s^-2
M_earth = 5.972e27  # g
R_earth = 6.371e8   # cm

# add planet gravity in cgs units (cm/s^2)
M_p = filtered_planets['pl_bmasse']
R_p = filtered_planets['pl_rade']

filtered_planets['g_planet_cgs'] = G * (M_p * M_earth) / ( (R_p * R_earth)**2 )

# Already computed g_planet_cgs
g = filtered_planets['g_planet_cgs']

# Mass and radius errors
M_err = filtered_planets['pl_bmasseerr_avg']
R_err = filtered_planets['pl_radeerr_avg']

# Fractional error
frac_g_err = np.sqrt( (M_err / filtered_planets['pl_bmasse'])**2 +
                      (2 * R_err / filtered_planets['pl_rade'])**2 )
# Absolute error in cm/s²
filtered_planets['g_planet_cgs_err'] = g * frac_g_err


# set size limits for filtering
radius_lower = 1.5 # earth radii
radius_upper = 4 

# filter for planets in size limits
filtered_sizes = filtered_planets[
    (filtered_planets['pl_rade'] >= radius_lower) &
    (filtered_planets['pl_rade'] <= radius_upper)
]

fs_with_Teq = filtered_sizes.copy()
fs_with_Teq['pl_Teq'] = calc_Teq(
    fs_with_Teq['st_teff'],
    fs_with_Teq['st_rad'],
    fs_with_Teq['pl_orbsmax'],
    A=0.3
)

#from tamzin's code for uncertainty calculation
#fractional error in Teq
frac_Teq_err = np.sqrt(
    (fs_with_Teq['st_tefferr_avg'] / fs_with_Teq['st_teff'])**2 +
    (0.5 * fs_with_Teq['st_raderr_avg'] / fs_with_Teq['st_rad'])**2 +
    (0.5 * fs_with_Teq['pl_orbsmaxerr_avg'] / fs_with_Teq['pl_orbsmax'])**2
)

fs_with_Teq['pl_Teq_err'] = fs_with_Teq['pl_Teq'] * frac_Teq_err


fs_with_TSM = fs_with_Teq.copy()
fs_with_TSM['pl_tsm'] = calc_TSM(
    fs_with_TSM['pl_rade'],
    fs_with_TSM['pl_bmasse'],
    fs_with_TSM['pl_Teq'],
    fs_with_TSM['st_rad'],
    fs_with_TSM['sy_jmag'],  # 2MASS J-mag
)


frac_TSM_err = np.sqrt(
    (3 * fs_with_TSM['pl_radeerr_avg'] / fs_with_TSM['pl_rade'])**2 +
    (fs_with_TSM['pl_Teq_err'] / fs_with_TSM['pl_Teq'])**2 +
    (fs_with_TSM['pl_bmasseerr_avg'] / fs_with_TSM['pl_bmasse'])**2 +
    (2 * fs_with_TSM['st_raderr_avg'] / fs_with_TSM['st_rad'])**2 +
    ((np.log(10) / 5) * fs_with_TSM['sy_jmagerr_avg'])**2
)

fs_with_TSM['pl_tsm_err'] = fs_with_TSM['pl_tsm'] * frac_TSM_err
planets = fs_with_TSM.copy()
# planets.to_csv('filtered_exoplanets.csv', index=False)   

In [4]:
# checks
print('initial downloaded table dimensions:', df.shape ,'\nand dimensions after filtering for usable:')
print('column:', useful_columns_only.shape)
print('mass:', default_planets_m.shape)
#print('radius:', default_planets_mr.shape)
print('desired parameters:', filtered_planets.shape)
print('size:', planets.shape)
#fs_with_TSM[['pl_name','pl_rade','pl_bmasse','pl_Teq','st_rad','sy_jmag','pl_tsm']].head()

initial downloaded table dimensions: (6080, 145) 
and dimensions after filtering for usable:
column: (6080, 33)
mass: (595, 33)
desired parameters: (337, 41)
size: (240, 45)


In [121]:
# planets.to_csv('planets.csv', index=False)
# list(planets.columns)

In [5]:
max_Teq = 500
min_st_teff = 3000

lowtempplanets = planets[
    (planets['pl_Teq'] < max_Teq) & 
    (planets['st_teff'] > min_st_teff)
]
print('size:', lowtempplanets.shape)

size: (47, 45)


In [6]:
highjmagplanets = lowtempplanets[
    (lowtempplanets['sy_jmag'] > 7)]

print('size after filtering for Jmag >7:', highjmagplanets.shape)

size after filtering for Jmag >7: (43, 45)


In [7]:
def select_top_planets(df, temp_col="pl_Teq", tsm_col="pl_tsm", top_n=10, key_col="pl_name"):
    """
    top_n = number per bin
    returns:
        selection_table: top N planets per bin with selected columns
        selected_full: full original rows of selected planets
    """
    df = df.copy()
    
    # Define temperature bins and labels
    temp_bins = [200, 250, 300, 350, 400, 450, 500]
    #temp_bins = [0, 200, 400, 600, 800, 1000]
    bin_labels = [
        "Cool (<200 K)", "Warm (200–400 K)", "Hot (400–600 K)",
        "Very hot (600–800 K)", "Extremely hot (800–1000 K)", 
    ]
    bin_labels = [
        "200-250 K", "250-300 K", "300-350 K", "350-400 K", "400-450 K", "450-500 K" 
    ]
    
    # Add temperature bin column
    df["Teq_bin"] = pd.cut(df[temp_col], bins=temp_bins, labels=bin_labels, right=False)
    
    # Sort by TSM descending
    df_sorted = df.sort_values(tsm_col, ascending=False)
    
    # Take top N per temperature bin
    top_per_bin = df_sorted.groupby("Teq_bin").head(top_n)
    
    # Keep only relevant columns for selection table
    selection_columns = [
        key_col, "pl_Teq", "pl_Teq_err",
        "pl_tsm", "pl_tsm_err", "Teq_bin", "pl_trandur", "sy_jmag"
    ]
    selection_table = top_per_bin[selection_columns] \
                        .sort_values(["Teq_bin", "pl_tsm"], ascending=[True, False])
    
    # Get full rows for selected planets
    selected_full = df[df[key_col].isin(selection_table[key_col])].copy()
    
    return selection_table, selected_full




N_sample_per_bin = 4
selection_table, selected_full = select_top_planets(highjmagplanets, top_n=N_sample_per_bin)


# Optional: view selection table
# list(selected_full.columns)


C:\Users\Stan\AppData\Local\Temp\ipykernel_9312\2662140712.py:28: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  top_per_bin = df_sorted.groupby("Teq_bin").head(top_n)


In [ ]:
from datetime import datetime
now = datetime.now()

# remove extra columns with words
selected_full_dropped = selected_full.drop(columns=['discoverymethod','Teq_bin'])

# Save full table
output_file = f"selected_planets_under_500K{now.strftime('%d.%m_%H-%M')}.csv"
selected_full_dropped.to_csv(output_file, index=False)
print(f"Saved {len(selected_full_dropped)} planets to {output_file}")

selected_full.head()

Saved 19 planets to selected_planets_under_500K03.02_14-42.csv


In [8]:
total = selected_full['pl_trandur'].sum()
print('Total transit duration (hours):', total)

Total transit duration (hours): 41.7261777


In [ ]:
#plt.figure(figsize=(7,5))
plt.figure()
sc = plt.scatter(
    planets['pl_bmasse'],
    planets['pl_rade'],
    c=planets['pl_Teq'],
    cmap='viridis',
    alpha=0.7
)

#
#plt.xscale('log')
plt.xlabel('Planet Mass (M⊕)')
plt.ylabel('Planet Radius (R⊕)')
plt.title('Planet Radius vs Planet Mass')
cbar = plt.colorbar(sc)
cbar.set_label('Planet Equilibrium Temperature (K)')
plt.show()


In [ ]:
#plt.figure(figsize=(7,5))
plt.figure()
sc = plt.scatter(
    lowtempplanets['pl_bmasse'],
    lowtempplanets['pl_rade'],
    c=lowtempplanets['pl_Teq'],
    cmap='viridis',
    alpha=0.7
)

#
#plt.xscale('log')
plt.xlabel('Planet Mass (M⊕)')
plt.ylabel('Planet Radius (R⊕)')
plt.title('Planet Radius vs Planet Mass (T=<1000K)')
cbar = plt.colorbar(sc)
cbar.set_label('Planet Equilibrium Temperature (K)')
plt.show()


In [ ]:
# row with maximum Teq
max_row = planets.loc[lowtempplanets['pl_Teq'].idxmax ()]

#max_row
# print("Max mass value:", max_row['pl_bmasse'])

In [ ]:
# Plotting TSM vs equilibrium temperature of selected table
plt.figure(figsize=(8,6))
plt.errorbar(
    selected_full['pl_Teq'], 
    selected_full['pl_tsm'], 
    xerr=selected_full['pl_Teq_err'], 
    yerr=selected_full['pl_tsm_err'], 
    fmt='o', 
    ecolor='gray', 
    alpha=0.7, 
    markersize=4, 
    capsize=2
)
plt.xlabel("Equilibrium Temperature (K)")
plt.ylabel("TSM")
plt.title("TSM vs Equilibrium Temperature (Log Scale) for Selected Planets")
#plt.yscale("log")
plt.grid(True, which="both", ls="--", lw=0.5, alpha=0.5)
plt.show()

In [ ]:
# Plotting TSM vs equilibrium temperature to check it looks reasonable
plt.figure(figsize=(8,6))
plt.errorbar(
    planets['pl_Teq'], 
    planets['pl_tsm'], 
    xerr=planets['pl_Teq_err'], 
    yerr=planets['pl_tsm_err'], 
    fmt='o', 
    ecolor='gray', 
    alpha=0.7, 
    markersize=4, 
    capsize=2
)
plt.xlabel("Equilibrium Temperature (K)")
plt.ylabel("TSM")
plt.title("TSM vs Equilibrium Temperature (Log Scale)")
plt.yscale("log")
plt.grid(True, which="both", ls="--", lw=0.5, alpha=0.5)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

planets['pl_tsm'].hist(bins=50) 
plt.xlabel('TSM')
plt.ylabel('Number of planets')
plt.title('Histogram of TSM')
plt.show()
# checked and this ignores NaNs automatically

In [ ]:
lowtempplanets['pl_Teq'].hist(bins=25) 
plt.xlabel('Planet equilibrium temperature (K)')
plt.ylabel('Number of planets')
plt.title('Histogram of Planet Equilibrium Temperatures 0-1000K')
plt.show()

In [ ]:
# checks
print('initial downloaded table dimensions:', df.shape ,'\nand dimensions after filtering for usable:')
print('column:', useful_columns_only.shape)
print('mass:', default_planets_m.shape)
print('radius:', default_planets_mr.shape)
print('desired parameters:', default_planets_mr.shape)
print('size:', lowtempplanets.shape)
#fs_with_TSM[['pl_name','pl_rade','pl_bmasse','pl_Teq','st_rad','sy_jmag','pl_tsm']].head()

In [ ]:
# row with maximum Teq
max_row = planets.loc[planets['pl_Teq'].idxmax()]


print("Max Teq value:", max_row['pl_Teq'])

# row with minimum Teq
min_row = planets.loc[planets['pl_Teq'].idxmin()]

print("Min Teq value:", min_row['pl_Teq'])


In [ ]:
tempcheck = planets[
    (planets['pl_Teq'] <= 1000)
]
print("Number of planets with Teq <= 1000 K:", tempcheck.shape[0])

In [ ]:
find_row = planets[planets['pl_name'] == 'K2-18 b']
print(find_row)

In [ ]:
sorted_teq = planets.sort_values(by='g_planet_cgs', ascending=True)
sorted_teq.head(10)


# planets.plot(x='pl_Teq', y='pl_tsm', kind='scatter')  # scatter plot

# plt.xlabel('pl_Teq')  # optional: label x-axis
# plt.ylabel('pl_tsm')  # optional: label y-axis
# plt.title('pl_tsm vs pl_Teq')  # optional: add title
# plt.show()
